### 🧠 What is Query Decomposition?
Query decomposition is the process of taking a complex, multi-part question and breaking it into simpler, atomic sub-questions that can each be retrieved and answered individually.

#### ✅ Why Use Query Decomposition?

- Complex queries often involve multiple concepts

- LLMs or retrievers may miss parts of the original question

- It enables multi-hop reasoning (answering in steps)

- Allows parallelism (especially in multi-agent frameworks)

In [1]:
from langchain.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

/Users/sauravmajumdar/Developer/AI/RAG/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Step 1: Load and embed the document
loader = TextLoader("langchain_crewai_dataset.txt")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(docs)

embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embedding)
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 4, "lambda_mult": 0.7})

In [20]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

llm=init_chat_model(model="openai:gpt-4o-mini")
llm

ChatOpenAI(profile={'max_input_tokens': 128000, 'max_output_tokens': 16384, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x3196627b0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x319f1b200>, root_client=<openai.OpenAI object at 0x31872c0e0>, root_async_client=<openai.AsyncOpenAI object at 0x319662810>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

In [ ]:
# Step 3: Query decomposition
# decomposition_prompt = PromptTemplate.from_template("""
# You are an AI assistant. Decompose the following complex question into 2 to 4 smaller sub-questions for better document retrieval.

# Question: "{question}"

# Sub-questions:
# """)
# decomposition_chain = decomposition_prompt | llm | StrOutputParser()

In [21]:
# Step 3: Query decomposition
decomposition_prompt = PromptTemplate.from_template("""
I’m going to ask you a question. I want you to decompose it into a series of subquestions. Each subquestion should be self-contained with all the information necessary to solve it.

Make sure not to decompose more than necessary or have any trivial subquestions - you’ll be evaluated on the simplicity, conciseness, and correctness of your decompositions as well as your final answer. You should wrap each subquestion in <sub q></sub q> tags. After each subquestion, you should answer the subquestion and put your subanswer in <sub a></sub a> tags.

 Once you have all the information you need to answer the question, output <FIN></FIN> tags.

example:
Question: What is Bitcoin?
<sub q>What is the purpose of Bitcoin?</sub q>
<sub a>Bitcoin serves as a decentralized digital currency.</sub a>
<sub q>What does decentralized mean?</sub q>
<sub a>Decentralized means it operates without a central authority or single administrator.</sub a>
<FIN>Bitcoin is a decentralized digital currency that operates without a central authority.</FIN>

Question: {question}
""")
decomposition_chain = decomposition_prompt | llm | StrOutputParser()

In [22]:
query = "How does LangChain use memory and agents compared to CrewAI?"
decomposition_question=decomposition_chain.invoke({"question": query})


In [23]:
print(decomposition_question)

<sub q>What is LangChain's approach to memory in AI applications?</sub q>  
<sub a>LangChain uses memory to store and recall information pertinent to user interactions and tasks, allowing for context-aware responses and improved conversation flow.</sub a>  

<sub q>How does LangChain incorporate agents in its architecture?</sub q>  
<sub a>LangChain incorporates agents as autonomous entities that can interact with external tools or APIs, execute tasks, and respond to user queries based on defined objectives.</sub a>  

<sub q>What does CrewAI offer in terms of memory management for AI?</sub q>  
<sub a>CrewAI implements memory management by allowing AI assistants to learn from interactions and retain insights, enhancing long-term user engagement and personalization.</sub a>  

<sub q>What is the role of agents in CrewAI's system?</sub q>  
<sub a>In CrewAI, agents are designed to collaborate and coordinate tasks among themselves, thus enabling a more flexible and dynamic assistance app

In [24]:
# Step 4: QA chain per sub-question
qa_prompt = PromptTemplate.from_template("""
Use the context below to answer the question.

Context:
{context}

Question: {input}
""")
qa_chain = create_stuff_documents_chain(llm=llm, prompt=qa_prompt)

In [25]:
# Step 5: Full RAG pipeline logic
def full_query_decomposition_rag_pipeline(user_query):
    # Decompose the query
    sub_qs_text = decomposition_chain.invoke({"question": user_query})
    sub_questions = [q.strip("-•1234567890. ").strip() for q in sub_qs_text.split("\n") if q.strip()]
    
    results = []
    for subq in sub_questions:
        docs = retriever.invoke(subq)
        result = qa_chain.invoke({"input": subq, "context": docs})
        results.append(f"Q: {subq}\nA: {result}")
    
    return "\n\n".join(results)

In [26]:
# Step 6: Run
query = "How does LangChain use memory and agents compared to CrewAI?"
final_answer = full_query_decomposition_rag_pipeline(query)
print("✅ Final Answer:\n")
print(final_answer)

✅ Final Answer:

Q: <sub q>What is memory in the context of LangChain and CrewAI?</sub q>
A: In the context of LangChain and CrewAI, memory refers to the capability of the system to retain and utilize information from previous interactions within a conversation or task. LangChain offers specific memory modules, such as ConversationBufferMemory and ConversationSummaryMemory, which enable the language model (LLM) to keep track of past conversation turns or summarize lengthy interactions to remain within token limits. This allows for enhanced awareness and context during interactions.

In conjunction with CrewAI, which supports role-based collaboration and works with LangChain's tools, memory facilitates more effective communication and decision-making within hybrid systems. Together, these memory features help maintain context and continuity, enabling agents to plan and execute tasks dynamically based on prior information.

Q: <sub a>Memory refers to the capability of a system to retain 